# Módulo de extracción, transformación y carga de datos

<p align='center'>
  <img width='200' heigth='225' src='https://user-images.githubusercontent.com/62605744/171186764-43f7aae0-81a9-4b6e-b4ce-af963564eafb.png'>
</p>

- Universidad del Valle - Escuela de ingeniería de sistemas y computación
- Asignatura: Introducción a la ciencia de los datos
- Semestre 2024-II
- Profesor: Oswaldo Solarte Pabon

ETL en Python para base de datos de mensajería

#### Autores
- Diego Fernando Victoria - 202125877 - diego.victoria@correounivalle.edu.co
- Janiert Sebastián Salas - 201941265 - janiert.salas@correounivalle.edu.co

#### Creación y activación de un entorno en Python
```
python3 -m venv my_env
```
```
source my_env/bin/activate  
```

#### Instalación de librerías necesarias
```
pip install -r requirements.txt
```

#### Creación y configuración del archivo config.yml
```
echo mensajeriaOLTP: {drivername: postgresql, host: localhost, port: 5432, dbname: (Nombre de la bd), user: (Nombre del usuario propietario), password: (Contraseña del usuario propietario)} mensajeriaOLAP: {drivername: postgresql, host: localhost, port: 5432, dbname: (Nombre de la bd), user: (Nombre del usuario propietario), password: (Contraseña del usuario propietario)} > config.yml
```

#### Importación de librerías

In [606]:
from datetime import date
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np
import holidays
import yaml
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

#### Conexión con las bases de datos

In [607]:
# Cargar configuraciones
with open('config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

# Crear URLs de conexión
url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")

# Crear las conexiones
oltp_conn, olap_conn = create_engine(url_oltp), create_engine(url_olap)

# Función para probar la conexión con las bases de datos
def comprobar_conexionBD(db_engine, db_name):
  try:
    with db_engine.connect() as connection:
      if connection.execute(text("SELECT 1")).fetchone() is not None:
        print(f"Conexión exitosa a base de datos {db_name}")
      else:
        print(f"No se pudo conectar a {db_name}")
  except SQLAlchemyError as e:
    print(f"Error al conectar a base de datos {db_name}: {e}")

comprobar_conexionBD(oltp_conn, config_oltp['dbname'])
comprobar_conexionBD(olap_conn, config_olap['dbname'])

Conexión exitosa a base de datos mensajeriaOLTP
Conexión exitosa a base de datos mensajeriaOLAP


## Dimensiones

### Dimensión cliente

#### Módulo de extracción

In [608]:
table_cliente = pd.read_sql_table('cliente', oltp_conn)
table_ciudad = pd.read_sql_table('ciudad', oltp_conn)
table_departamento = pd.read_sql_table('departamento', oltp_conn)
table_tipo_cliente = pd.read_sql_table('tipo_cliente', oltp_conn)

table_ciudad.head()

,ciudad_id,nombre,departamento_id
0,6,BUGA,1
1,5,BOGOTA,2
2,4,PASTO,4
3,3,POPAYAN,3
4,2,PALMIRA,1


#### Módulo de transformación

In [609]:
table_departamento = table_departamento.rename(columns={'nombre' : 'departamento'})
table_ciudad = (pd.merge(table_ciudad.rename(columns={'nombre' : 'ciudad'}), table_departamento, how='inner')
  .drop(columns=['departamento_id']))
table_tipo_cliente = (table_tipo_cliente.drop(columns=['descripcion']).rename(columns={'nombre': 'tipo_cliente'}))

dimension_cliente = pd.merge(table_cliente, table_ciudad, how='inner')
dimension_cliente = (pd.merge(dimension_cliente, table_tipo_cliente, how='inner')
  .drop(columns=['ciudad_id', 'tipo_cliente_id', 'coordinador_id', 'activo']))

dimension_cliente.set_index('cliente_id', inplace=True)

dimension_cliente.head()

,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,sector,ciudad,departamento,tipo_cliente
cliente_id,,,,,,,,,,
1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,S,CALI,VALLE DEL CAUCA,Persona Juridica
2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,industrial,CALI,VALLE DEL CAUCA,Persona Juridica
6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,salud,CALI,VALLE DEL CAUCA,Persona Juridica
19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,salud,CALI,VALLE DEL CAUCA,Persona Juridica
8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,salud,CALI,VALLE DEL CAUCA,Persona Juridica


#### Módulo de carga

In [610]:
try:
  dimension_cliente.to_sql('dim_cliente', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")

### Dimensión fecha

#### Módulo de extracción

In [611]:
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn, columns=['fecha', 'hora'])
table_novedadesservicio = pd.read_sql_table('mensajeria_novedadesservicio', oltp_conn, columns=['fecha_novedad'])
table_novedadesservicio['fecha_hora'] = pd.to_datetime(table_novedadesservicio['fecha_novedad'].dt.strftime('%Y-%m-%d %H:%M'))

table_novedadesservicio['fecha'] = table_novedadesservicio['fecha_novedad'].dt.date
table_novedadesservicio['hora'] = table_novedadesservicio['fecha_novedad'].dt.strftime('%H:%M') # intercambiar cambiar el formato
table_estadosservicio['hora'] = table_estadosservicio['hora'].astype(str).str[:5]
table_estadosservicio['fecha_hora'] = pd.to_datetime(table_estadosservicio['fecha'].astype(str) + ' ' + table_estadosservicio['hora'].astype(str))

#### Módulo de transformación

In [612]:
dimension_fecha = pd.concat([table_estadosservicio, table_novedadesservicio.drop(columns=['fecha_novedad'])])
dimension_fecha['fecha'] = pd.to_datetime(dimension_fecha['fecha'])

dimension_fecha["día"] = dimension_fecha["fecha"].dt.day
dimension_fecha["mes"] = dimension_fecha["fecha"].dt.month
dimension_fecha["año"] = dimension_fecha["fecha"].dt.year
dimension_fecha["día_semana"] = dimension_fecha["fecha"].dt.weekday
dimension_fecha["trimestre"] = dimension_fecha["fecha"].dt.quarter

dimension_fecha.head()

,fecha,hora,fecha_hora,día,mes,año,día_semana,trimestre
0,2024-01-29,01:13,2024-01-29 01:13:00,29,1,2024,0,1
1,2024-01-30,18:45,2024-01-30 18:45:00,30,1,2024,1,1
2,2024-02-06,11:34,2024-02-06 11:34:00,6,2,2024,1,1
3,2024-02-01,14:50,2024-02-01 14:50:00,1,2,2024,3,1
4,2024-04-06,16:11,2024-04-06 16:11:00,6,4,2024,5,2


In [613]:
dimension_fecha["día_del_año"] = dimension_fecha["fecha"].dt.day_of_year
dimension_fecha["nombre_mes"] = dimension_fecha["fecha"].dt.strftime('%B')
dimension_fecha["nombre_día"] = dimension_fecha["fecha"].dt.day_name()
dimension_fecha['nombre_día'] = dimension_fecha["nombre_día"].replace({'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles', 'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'})

dimension_fecha.head()

,fecha,hora,fecha_hora,día,mes,año,día_semana,trimestre,día_del_año,nombre_mes,nombre_día
0,2024-01-29,01:13,2024-01-29 01:13:00,29,1,2024,0,1,29,enero,lunes
1,2024-01-30,18:45,2024-01-30 18:45:00,30,1,2024,1,1,30,enero,martes
2,2024-02-06,11:34,2024-02-06 11:34:00,6,2,2024,1,1,37,febrero,martes
3,2024-02-01,14:50,2024-02-01 14:50:00,1,2,2024,3,1,32,febrero,jueves
4,2024-04-06,16:11,2024-04-06 16:11:00,6,4,2024,5,2,97,abril,sábado


In [614]:
co_holidays = holidays.CO(language="es")
dimension_fecha["es_festivo"] = dimension_fecha["fecha"].apply(lambda x:  x in co_holidays)
dimension_fecha["festivo"] = dimension_fecha["fecha"].apply(lambda x: co_holidays.get(x))
dimension_fecha["fin_de_semana"] = dimension_fecha["día_semana"].apply(lambda x: x>4)
dimension_fecha['fecha'] = dimension_fecha['fecha'].dt.date
dimension_fecha.head()

,fecha,hora,fecha_hora,día,mes,año,día_semana,trimestre,día_del_año,nombre_mes,nombre_día,es_festivo,festivo,fin_de_semana
0,2024-01-29,01:13,2024-01-29 01:13:00,29,1,2024,0,1,29,enero,lunes,False,None,False
1,2024-01-30,18:45,2024-01-30 18:45:00,30,1,2024,1,1,30,enero,martes,False,None,False
2,2024-02-06,11:34,2024-02-06 11:34:00,6,2,2024,1,1,37,febrero,martes,False,None,False
3,2024-02-01,14:50,2024-02-01 14:50:00,1,2,2024,3,1,32,febrero,jueves,False,None,False
4,2024-04-06,16:11,2024-04-06 16:11:00,6,4,2024,5,2,97,abril,sábado,False,None,True


In [615]:
dimension_fecha['fecha'] = pd.to_datetime(dimension_fecha['fecha']).dt.date
dimension_fecha['hora'] = pd.to_datetime(dimension_fecha['hora'].astype(str).str.split('.').str[0], format='%H:%M').dt.time

dimension_fecha.drop_duplicates(subset=['fecha', 'hora'], inplace=True)

dimension_fecha.reset_index(drop=True, inplace=True)
dimension_fecha['id'] = dimension_fecha.index
dimension_fecha.set_index('id', inplace=True)

#### Módulo de carga

In [616]:
try:
  dimension_fecha.to_sql('dim_fecha', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")

### Dimensión mensajero

#### Módulo de extracción

In [617]:
tabla_mensajero = pd.read_sql_table('clientes_mensajeroaquitoy', oltp_conn)
tabla_user = pd.read_sql_table('auth_user', oltp_conn)
tabla_ciudad = pd.read_sql_table('ciudad', oltp_conn)
tabla_departamento = pd.read_sql_table('departamento', oltp_conn)

#### Módulo de transformación

In [618]:
dimension_mensajero = (tabla_mensajero[['id', 'user_id', 'activo', 'fecha_entrada', 'fecha_salida', 'salario', 'telefono', 'ciudad_operacion_id']]
  .rename(columns={'id' : 'id_mensajero'}))

tabla_ciudad = (pd.merge(tabla_ciudad.rename(columns={'nombre' : 'ciudad'}), tabla_departamento.rename(columns={'nombre' : 'departamento'}), how='inner')
  .drop(columns=['departamento_id'])) 

dimension_mensajero = (pd.merge(dimension_mensajero, tabla_ciudad, 
  how='left', left_on='ciudad_operacion_id', right_on='ciudad_id')
  .drop(columns=['ciudad_operacion_id', 'ciudad_id']))

tabla_user.drop(columns=['password', 'is_superuser', 'is_staff', 'date_joined', 'last_login'], inplace=True)

dimension_mensajero = (pd.merge(dimension_mensajero, tabla_user, 
  how='inner', left_on='user_id', right_on='id')
  .drop(columns=['user_id', 'id', 'activo', 'is_active', 'fecha_salida']))

dimension_mensajero['fecha_entrada'] = dimension_mensajero['fecha_entrada'].dt.date
dimension_mensajero.set_index('id_mensajero', inplace=True)
dimension_mensajero.sort_values(by=['id_mensajero']).head()

,fecha_entrada,salario,telefono,ciudad,departamento,username,first_name,last_name,email
id_mensajero,,,,,,,,,
1,NaT,NaN,310-300000,ACOPI YUMBO,VALLE DEL CAUCA,mensajero1,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,NaT,NaN,310-300000,NaN,NaN,mensajero2,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,2012-05-08,3000000.0,310-300000,CALI,VALLE DEL CAUCA,Biil-Gates,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,2018-12-17,1160000.0,310-300000,CALI,VALLE DEL CAUCA,Lionel_messi,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
5,2015-07-01,1160000.0,310-300000,NaN,NaN,James Rodriguez,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


#### Módulo de carga

In [619]:
try:
  dimension_mensajero.to_sql('dim_mensajero', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")

### Dimensión sede

#### Módulo de extracción

In [620]:
table_sede = pd.read_sql_table('sede', oltp_conn)
table_ciudad = pd.read_sql_table('ciudad', oltp_conn)
table_departamento = pd.read_sql_table('departamento', oltp_conn)

#### Módulo de transformación

In [621]:
table_ciudad = (pd.merge(table_ciudad.rename(columns={'nombre' : 'ciudad'}), table_departamento.rename(columns={'nombre' : 'departamento'}), how='inner')
  .drop(columns=['departamento_id']))

dimension_sede = (pd.merge(table_sede.rename(columns={'nombre' : 'nombre_sede'}), table_ciudad, 
  how='inner', left_on='ciudad_id', right_on='ciudad_id')
  .drop(columns=['ciudad_id', 'cliente_id'])
  .sort_values(by='sede_id'))

dimension_sede.set_index('sede_id', inplace=True)

dimension_sede.head()

,nombre_sede,direccion,telefono,nombre_contacto,ciudad,departamento
sede_id,,,,,,
1,Sede principal - Cliente1,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
2,sede aux - cliente 1,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
3,TORRES DE MARACAIBO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
4,INGENIO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
5,VASQUEZ COBO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA


#### Módulo de carga

In [622]:
try:
  dimension_sede.to_sql('dim_sede', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")

## Hechos

### Hecho novedades

#### Módulo de extracción

In [623]:
table_novedadesservicio = pd.read_sql_table('mensajeria_novedadesservicio', oltp_conn)
table_tipo_novedades = pd.read_sql_table('mensajeria_tiponovedad', oltp_conn)
dimension_fecha = pd.read_sql_table('dim_fecha', olap_conn)
dimension_mensajero = pd.read_sql_table('dim_mensajero', olap_conn)

#### Módulo de transformación

In [624]:
table_tipo_novedades.sort_values(by=['id'], inplace=True)
table_novedadesservicio['fecha_novedad'] = pd.to_datetime(table_novedadesservicio['fecha_novedad'].dt.strftime('%Y-%m-%d %H:%M'))

hecho_novedades = (pd.merge(
  table_novedadesservicio[['fecha_novedad', 'tipo_novedad_id']], 
  dimension_fecha[['fecha_hora', 'id', 'fecha']],
  how='left', left_on='fecha_novedad', right_on='fecha_hora')
  .drop(columns=['fecha_novedad', 'fecha_hora'])
  .rename(columns={'id': 'id_fecha'}))

conteo_novedades = {}

for tipo_id in table_tipo_novedades['id']: # genera todas las llaves con la dimensión fecha 
  conteo_novedades[f'num_novedades_{tipo_id}'] = (hecho_novedades[hecho_novedades['tipo_novedad_id'] == tipo_id]
    .groupby('fecha')['tipo_novedad_id']
    .count())

# conteo total por tipo novedad
conteo_novedades = pd.DataFrame(conteo_novedades).fillna(0).astype(int)
# num_novedades como la suma de los conteos por tipo
conteo_novedades['total_novedades'] = conteo_novedades.sum(axis=1)

hecho_novedades = (hecho_novedades.merge(conteo_novedades, on='fecha', how='left')
  .drop_duplicates(subset=['fecha'])
  .drop(columns=['fecha', 'tipo_novedad_id'])
  .reset_index(drop=True))

#hecho_novedades.head()
#print(hecho_novedades.shape[0])

#### Módulo de carga

In [625]:
try:
  hecho_novedades.to_sql('hecho_novedades', olap_conn, if_exists='replace', index_label='id')
except Exception as e:
  print(f"Error al cargar datos: {e}")

### Hecho servicios

#### Módulo de extracción

In [626]:
table_servicio = pd.read_sql_table('mensajeria_servicio', oltp_conn)
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn)
table_usuario = pd.read_sql_table('clientes_usuarioaquitoy', oltp_conn)
dimension_fecha = pd.read_sql('dim_fecha', olap_conn)

estados = [
  (1, 'id_fecha_iniciado'),
  (2, 'id_fecha_mensajero'),
  (3, 'id_fecha_recogida'),
  (4, 'id_fecha_entrega'),
  (5, 'id_fecha_terminado')]

#### Módulo de transformación

In [627]:
hecho_servicios = (table_servicio[['id', 'mensajero_id', 'mensajero2_id', 'mensajero3_id', 'usuario_id']]
  .rename(columns={'mensajero_id' : 'mensajero1_id'})
  .astype('Int32')
  .replace({pd.NA: None}))
hecho_servicios = (pd.merge(hecho_servicios, table_usuario[['id', 'cliente_id', 'sede_id']].rename(columns={'id' : 'usuario_id'}), 
  how='left', on='usuario_id'))

In [628]:
# selecciona el último mensajero que prestó el servicio
hecho_servicios['mensajero_id'] = (hecho_servicios[['mensajero3_id', 'mensajero2_id', 'mensajero1_id']]
  .astype('Int32')
  .replace({pd.NA: None})
  .bfill(axis=1)
  .infer_objects(copy=False)
  .iloc[:, 0])

hecho_servicios.drop(columns=['mensajero3_id', 'mensajero2_id', 'mensajero1_id'], inplace=True)

In [629]:
dimension_fecha['hora'] = dimension_fecha['hora'].astype(str).str[:5]

for estado_id, nombre in estados: # genera todas las llaves con la dimensión fecha 
  filtered_estados = (table_estadosservicio[['estado_id', 'servicio_id', 'fecha', 'hora']]
    .query(f"estado_id == {estado_id}")
    .drop(columns=['estado_id'])
    .drop_duplicates(subset=['servicio_id']))
  
  filtered_estados['hora'] = filtered_estados['hora'].astype(str).str[:5]

  filtered_estados = (pd.merge(filtered_estados, dimension_fecha[['fecha', 'hora', 'id']],
    how='left', 
    on=['fecha', 'hora'])
    .rename(columns={'id': nombre})
    .drop(columns=['fecha', 'hora']))

  hecho_servicios = (hecho_servicios
    .merge(filtered_estados, how='left', left_on='id', right_on='servicio_id')
    .drop(columns=['servicio_id'])
    .astype('Int32')
    .replace({pd.NA: None}))

for i in range(len(estados) - 1):
  estado_actual = estados[i][1]
  estado_siguiente = estados[i + 1][1]
    
  tiempos_estados = hecho_servicios[['id', estado_actual, estado_siguiente]].rename(columns={'id': 'id_servicio'})
    
  tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
    how='left', left_on=estado_actual, right_on='id')
    .rename(columns={'fecha_hora': estado_actual.replace("id_", "")})
    .drop(columns=['id', estado_actual]))

  tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
    how='left', left_on=estado_siguiente, right_on='id')
    .rename(columns={'fecha_hora': estado_siguiente.replace("id_", "")})
    .drop(columns=['id', estado_siguiente]))

  # diferencia de tiempo entre los pares de estados
  tiempos_estados['diferencia_tiempo'] = tiempos_estados[estado_siguiente.replace("id_", "")] - tiempos_estados[estado_actual.replace("id_", "")]

  # diferencia en horas y minutos
  tiempos_estados[f'horas{estado_actual.replace("id_fecha", "")}_{estado_siguiente.replace("id_fecha_", "")}'] = ((tiempos_estados['diferencia_tiempo'].dt.total_seconds() / 3600)
    .round()
    .astype('Int32')
    .replace({pd.NA: None}))
    
  tiempos_estados.drop(columns=[estado_actual.replace("id_", ""), estado_siguiente.replace("id_", ""), 'diferencia_tiempo'])

  hecho_servicios = (pd.merge(hecho_servicios, tiempos_estados.drop(columns=[estado_actual.replace("id_", ""), estado_siguiente.replace("id_", ""), 'diferencia_tiempo']),
    how='left', left_on='id', right_on='id_servicio')
    .drop(columns=['id_servicio']))

tiempos_estados = hecho_servicios[['id', estados[0][1], estados[len(estados) - 1][1]]]

tiempos_estados = (pd.merge(tiempos_estados.rename(columns={'id' : 'id_servicio'}), dimension_fecha[['id', 'fecha_hora']], 
  how='inner', left_on=estados[0][1], right_on='id')
  .drop(columns=['id_fecha_iniciado', 'id'])
  .rename(columns={'fecha_hora' : 'fecha_hora_iniciado'}))

tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
  how='inner', left_on='id_fecha_terminado', right_on='id')
  .drop(columns=['id_fecha_terminado', 'id'])
  .rename(columns={'fecha_hora' : 'fecha_hora_terminado'}))

tiempos_estados['horas_iniciado_terminado'] = (
  ((tiempos_estados['fecha_hora_terminado'] - tiempos_estados['fecha_hora_iniciado']).dt.total_seconds() / 3600)
  .round()
  .astype('Int32'))

hecho_servicios = (pd.merge(hecho_servicios, tiempos_estados.drop(columns=['fecha_hora_iniciado', 'fecha_hora_terminado']), 
  how='left', left_on='id', right_on='id_servicio')
  .drop(columns=['id_servicio'])
  .replace({pd.NA: None}))

#print(hecho_servicios.shape[0])
#hecho_servicios.sort_values(by=['id']).head()

#### Módulo de carga

In [630]:
try:
  hecho_servicios.set_index('id').to_sql('hecho_servicios', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")

## Solución preguntas

### Pregunta 1
¿En qué meses del año los clientes solicitan más servicios de mensajería?

In [631]:
# Mes con más servicios en general (todos los años)
hecho_servicios_preguntas = hecho_servicios[['id', 'id_fecha_iniciado']].rename(columns={'id' : 'id_servicio'})

hecho_servicios_preguntas = (pd.merge(hecho_servicios_preguntas, dimension_fecha[['id', 'año','mes']].astype('Int32'), 
  how='left', left_on='id_fecha_iniciado', right_on='id').drop(columns=['id', 'id_fecha_iniciado'])) 

hecho_servicios_preguntas = (hecho_servicios_preguntas.groupby('mes')['id_servicio']
  .count()
  .reset_index()
  .rename(columns={'id_servicio': 'num_servicios'}))

hecho_servicios_preguntas['mes'] = hecho_servicios_preguntas['mes'].map({ 1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre' })

hecho_servicios_preguntas.sort_values(by=['num_servicios'], ascending=False)

,mes,num_servicios
4,Mayo,4725
6,Julio,4549
3,Abril,4480
7,Agosto,4304
5,Junio,4184
2,Marzo,3337
1,Febrero,2479
0,Enero,296
11,Diciembre,25
8,Septiembre,21


### Pregunta 2
¿Cuáles son los días donde más solicitudes hay?

In [632]:
# Servicios por día de la semana global (todos los años)
hecho_servicios_preguntas = hecho_servicios[['id', 'id_fecha_iniciado']].rename(columns={'id' : 'id_servicio'})

hecho_servicios_preguntas = (pd.merge(hecho_servicios_preguntas, dimension_fecha[['id','día_semana']].astype(int), 
  how='left', left_on='id_fecha_iniciado', right_on='id')
  .drop(columns=['id', 'id_fecha_iniciado']))

# Agrupa por 'día_semana' y cuenta la cantidad de servicios
hecho_servicios_preguntas = (hecho_servicios_preguntas
  .groupby('día_semana')['id_servicio']
  .count()
  .reset_index()
  .rename(columns={'id_servicio': 'num_servicios'}))

hecho_servicios_preguntas['día_semana'] = (hecho_servicios_preguntas['día_semana']
  .replace({0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}))

hecho_servicios_preguntas.sort_values(by=['num_servicios'], ascending=False)

,día_semana,num_servicios
1,Martes,5397
4,Viernes,5282
3,Jueves,5158
2,Miércoles,4962
0,Lunes,4308
5,Sábado,2482
6,Domingo,840


### Pregunta 3
¿A qué hora los mensajeros están más ocupados?

In [633]:
# Hora en que los mensajeros están más ocupados (se asume la hora de más ocupado como la hora de recogida) global
hecho_servicios_preguntas = hecho_servicios[['id', 'id_fecha_recogida']].rename(columns={'id' : 'id_servicio'})

hecho_servicios_preguntas = (pd.merge(hecho_servicios_preguntas, dimension_fecha[['id', 'hora']], 
  how='inner', left_on='id_fecha_recogida', right_on='id')
  .drop(columns=['id', 'id_fecha_recogida']))

hecho_servicios_preguntas['hora'] = pd.to_datetime(hecho_servicios_preguntas['hora'], format='%H:%M').dt.hour

hecho_servicios_preguntas = (hecho_servicios_preguntas.groupby('hora')['id_servicio']
  .count()
  .reset_index()
  .rename(columns={'id_servicio': 'num_servicios'}))

hecho_servicios_preguntas.sort_values(by='num_servicios', ascending=False).head()

,hora,num_servicios
10,11,374
15,16,368
9,10,358
8,9,357
14,15,331


### Pregunta 4
Número de servicios solicitados por cliente y por mes?

In [634]:
# cantidad de servicios solicitados por cliente
hecho_servicios_preguntas = (hecho_servicios.groupby('cliente_id')['id']
  .count()
  .reset_index()
  .rename(columns={'id': 'num_servicios'}))

#print(conteo_servicios_por_cliente.shape[0])
print("Servicios solicitados por cliente:")
print(hecho_servicios_preguntas)

# Mes con más servicios en general (todos los años)
hecho_servicios_preguntas = hecho_servicios[['id', 'id_fecha_iniciado']].rename(columns={'id' : 'id_servicio'})

hecho_servicios_preguntas = (pd.merge(hecho_servicios_preguntas, dimension_fecha[['id', 'año','mes']].astype('Int32'), 
  how='left', left_on='id_fecha_iniciado', right_on='id').drop(columns=['id', 'id_fecha_iniciado'])) 

hecho_servicios_preguntas = (hecho_servicios_preguntas.groupby('mes')['id_servicio']
  .count()
  .reset_index()
  .rename(columns={'id_servicio': 'num_servicios'}))

hecho_servicios_preguntas['mes'] = hecho_servicios_preguntas['mes'].map({ 1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre' })

print("\nServicios solicitados por mes:")
print(hecho_servicios_preguntas.sort_values(by=['num_servicios'], ascending=False))

Servicios solicitados por cliente:
    cliente_id  num_servicios
0            2            104
1            3            205
2            4            117
3            5           4578
4            6            292
5            7           2290
6            8            296
7            9            656
8           11          17384
9           12           1409
10          22             62
11          24             28
12          25           1008
13          27              1

Servicios solicitados por mes:
           mes  num_servicios
4         Mayo           4725
6        Julio           4549
3        Abril           4480
7       Agosto           4304
5        Junio           4184
2        Marzo           3337
1      Febrero           2479
0        Enero            296
11   Diciembre             25
8   Septiembre             21
10   Noviembre             17
9      Octubre             12


### Pregunta 5
Mensajeros más eficientes (Los que más servicios prestan)

In [635]:
# mensajero que más servicios prestó de forma global

# Cuenta las apariciones de cada mensajero
hecho_servicios_preguntas = hecho_servicios['mensajero_id'].value_counts().reset_index()
hecho_servicios_preguntas.columns = ['mensajero_id', 'numero_servicios']

hecho_servicios_preguntas = hecho_servicios_preguntas.astype('Int32').sort_values(by='numero_servicios', ascending=False)

hecho_servicios_preguntas.head()

,mensajero_id,numero_servicios
0,30,2419
1,29,1527
2,15,1522
3,25,1420
4,31,1346


### Pregunta 6
¿Cuáles son las sedes que más servicios solicitan por cada cliente?

In [636]:
# número de servicios por cada sede de cada cliente
hecho_servicios_preguntas = (hecho_servicios.groupby(['usuario_id', 'cliente_id', 'sede_id'])['id']
  .count()
  .reset_index()
  .rename(columns={'id': 'num_servicios'})
  # Ordena primero por 'cliente_id' ascendente, luego por 'num_servicios' descendente
  .sort_values(by=['cliente_id', 'num_servicios'], ascending=[True, False]))

hecho_servicios_preguntas.drop(columns=['usuario_id'])

,cliente_id,sede_id,num_servicios
83,2,2,104
21,3,38,205
11,4,11,83
12,4,10,30
13,4,14,3
...,...,...,...
97,25,45,295
99,25,45,205
100,25,44,31
98,25,44,25


### Pregunta 7
¿Cuál es el tiempo promedio de entrega desde que se solicita el servicio hasta que se cierra el caso?

In [637]:
print("El tiempo promedio desde que se inicia un servicio hasta que se completa es de", round(hecho_servicios['horas_iniciado_terminado'].mean(), 2), "horas")

El tiempo promedio desde que se inicia un servicio hasta que se completa es de 5.83 horas


### Pregunta 8
Mostrar los tiempos de espera por cada fase del servicio: Iniciado, Con mensajero asignado, recogido en origen, Entregado en Destino, Cerrado. En que fase del servicio hay más demoras?

In [638]:
pd.set_option('future.no_silent_downcasting', True)

hecho_servicios_preguntas = hecho_servicios.copy()

hecho_servicios_preguntas[['horas_iniciado_mensajero', 'horas_mensajero_recogida', 'horas_recogida_entrega', 'horas_entrega_terminado']] = (
  hecho_servicios_preguntas[['horas_iniciado_mensajero', 'horas_mensajero_recogida', 'horas_recogida_entrega', 'horas_entrega_terminado']]
  .replace({None: np.nan})
  .infer_objects(copy=False)
  .where(lambda x: x >= 0, np.nan)
  .fillna(0) # reemplaza NaN por 0 en las columnas
)

# Calcula el promedio de cada columna
promedios = hecho_servicios_preguntas[['horas_iniciado_mensajero', 'horas_mensajero_recogida', 'horas_recogida_entrega', 'horas_entrega_terminado']].mean()

print(f"Horas promedios de cada columna:\n{promedios}")
print(f"La columna con mayor promedio de demora es '{promedios.idxmax()}' con un promedio de {promedios.max():.2f} horas.")
hecho_servicios_preguntas[['horas_iniciado_mensajero', 'horas_mensajero_recogida', 'horas_recogida_entrega', 'horas_entrega_terminado']]

Horas promedios de cada columna:
horas_iniciado_mensajero    2.366690
horas_mensajero_recogida    0.279071
horas_recogida_entrega      0.151882
horas_entrega_terminado     1.660359
dtype: float64
La columna con mayor promedio de demora es 'horas_iniciado_mensajero' con un promedio de 2.37 horas.


,horas_iniciado_mensajero,horas_mensajero_recogida,horas_recogida_entrega,horas_entrega_terminado
0,0.0,0.0,0.0,0.0
1,51.0,0.0,0.0,942.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
28425,0.0,0.0,0.0,0.0
28426,0.0,0.0,0.0,1.0
28427,0.0,0.0,0.0,1.0
28428,0.0,0.0,0.0,0.0


### Pregunta 9
¿Cuáles son las novedades que más se presentan durante la prestación del servicio?

In [639]:
pd.read_sql_table('hecho_novedades', olap_conn)

,id,id_fecha,num_novedades_1,num_novedades_2,total_novedades
0,0,76437,6,3,9
1,1,76438,2,1,3
2,2,76439,0,1,1
3,3,76440,1,3,4
4,4,76443,2,1,3
...,...,...,...,...,...
198,198,74498,62,8,70
199,199,79135,19,6,25
200,200,79150,70,10,80
201,201,75994,41,9,50
